In [2]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from dataclasses import dataclass
from datetime import datetime
from typing import Dict, List, Optional, Tuple
import pandas as pd

In [3]:
SEAT_COUNT = 16

@dataclass
class Reservation:
    rid: int
    name: str
    size: int
    created_at: str
    seats: List[int]  # 1-based contiguous seats

reservations: Dict[int, Reservation] = {}
next_rid = 1

event_log: List[dict] = []  # newest first


def compact_assign(res_map: Dict[int, Reservation]) -> Dict[int, List[int]]:
    """
    Always compact from seat 1 with NO gaps.
    Stable policy: keep booking order by rid.
    """
    cursor = 1
    out: Dict[int, List[int]] = {}
    for rid in sorted(res_map.keys()):
        sz = res_map[rid].size
        if cursor + sz - 1 > SEAT_COUNT:
            raise ValueError("Over capacity; cannot compact.")
        out[rid] = list(range(cursor, cursor + sz))
        cursor += sz
    return out


def seat_map_from_assign(assign: Dict[int, List[int]]) -> List[Optional[int]]:
    sm = [None] * SEAT_COUNT
    for rid, seats in assign.items():
        for s in seats:
            sm[s - 1] = rid
    return sm


def render_seat_map(sm: List[Optional[int]]) -> str:
    parts = []
    for i, rid in enumerate(sm, start=1):
        parts.append(f"{i:02d}[  ]" if rid is None else f"{i:02d}[{rid:02d}]")
    return " ".join(parts)


def free_block(sm: List[Optional[int]]) -> Tuple[int, int, int]:
    """Return first free contiguous block as (start,end,length). If none -> (0,0,0)."""
    n = len(sm)
    i = 0
    while i < n and sm[i] is not None:
        i += 1
    if i == n:
        return (0, 0, 0)
    start = i + 1
    j = i
    while j < n and sm[j] is None:
        j += 1
    end = j
    return (start, end, end - start + 1)


def moves_between(before: Dict[int, List[int]], after: Dict[int, List[int]]) -> List[dict]:
    moves = []
    for rid in sorted(set(before.keys()) | set(after.keys())):
        old = before.get(rid)
        new = after.get(rid)
        if old != new:
            moves.append({"rid": rid, "old": old, "new": new})
    return moves


def log_event(action: str, meta: dict, before_assign: Dict[int, List[int]], after_assign: Dict[int, List[int]]):
    before_sm = seat_map_from_assign(before_assign)
    after_sm = seat_map_from_assign(after_assign)
    mv = moves_between(before_assign, after_assign)

    event_log.insert(0, {
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "action": action,
        "meta": meta,
        "before_assign": before_assign,
        "after_assign": after_assign,
        "before_sm": before_sm,
        "after_sm": after_sm,
        "moves": mv,
    })


def recompact_apply(action: str, meta: dict):
    before_assign = {rid: r.seats[:] for rid, r in reservations.items()}
    after_assign = compact_assign(reservations)

    # apply
    for rid, seats in after_assign.items():
        reservations[rid].seats = seats

    log_event(action, meta, before_assign, after_assign)


def remaining_seats() -> int:
    return SEAT_COUNT - sum(r.size for r in reservations.values())


def book(name: str, size: int) -> Tuple[bool, str]:
    global next_rid
    name = (name or "").strip() or "Walk-in"
    size = int(size)

    if size < 1 or size > SEAT_COUNT:
        return False, f"Party size must be 1–{SEAT_COUNT}."
    if remaining_seats() < size:
        return False, f"Not enough seats remaining. Remaining: {remaining_seats()}"

    rid = next_rid
    next_rid += 1
    reservations[rid] = Reservation(
        rid=rid,
        name=name,
        size=size,
        created_at=datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        seats=[]
    )
    recompact_apply("BOOK", {"rid": rid, "name": name, "size": size})
    return True, f"Booked #{rid} ({name}) party of {size}."


def cancel(rid: int) -> Tuple[bool, str]:
    rid = int(rid)
    if rid not in reservations:
        return False, "Reservation not found."
    meta = {"rid": rid, "name": reservations[rid].name, "size": reservations[rid].size}
    del reservations[rid]
    recompact_apply("CANCEL", meta)
    return True, f"Canceled #{rid}."


def reset_all():
    global reservations, next_rid, event_log
    reservations = {}
    next_rid = 1
    event_log = []

In [ ]:
# ---------- Widgets ----------
# Customer tab controls
cust_name = widgets.Text(value="", placeholder="e.g., Alex / Party Lee", description="Name:", layout=widgets.Layout(width="420px"))
cust_size = widgets.BoundedIntText(value=2, min=1, max=SEAT_COUNT, description="Party size:", layout=widgets.Layout(width="220px"))
book_btn = widgets.Button(description="Book (auto-compact)", button_style="success")

cancel_dropdown = widgets.Dropdown(options=[("— none —", None)], description="Cancel:", layout=widgets.Layout(width="520px"))
cancel_btn = widgets.Button(description="Cancel selected", button_style="warning")

reset_btn = widgets.Button(description="Reset all", button_style="danger")

cust_out = widgets.Output()

# Internal tab outputs
internal_out = widgets.Output()

# Tabs
tab = widgets.Tab()
tab.children = [widgets.VBox([]), widgets.VBox([])]
tab.set_title(0, "Customer Booking Page")
tab.set_title(1, "Internal System View")


def reservation_label(r: Reservation) -> str:
    seats_txt = f"{r.seats[0]}–{r.seats[-1]}" if r.seats else ""
    return f"#{r.rid} | {r.name} | size {r.size} | seats {seats_txt}"


def refresh_cancel_dropdown():
    if not reservations:
        cancel_dropdown.options = [("— none —", None)]
        cancel_dropdown.value = None
        return
    opts = [("— choose —", None)]
    for r in sorted(reservations.values(), key=lambda x: x.rid):
        opts.append((reservation_label(r), r.rid))
    cancel_dropdown.options = opts
    cancel_dropdown.value = None


def show_customer_view(message: str = ""):
    with cust_out:
        clear_output()
        sm = seat_map_from_assign({rid: r.seats[:] for rid, r in reservations.items()})
        fb = free_block(sm)

        if message:
            print(message)
            print()

        print(f"Seats remaining: {remaining_seats()} / {SEAT_COUNT}")
        if fb[2] > 0:
            print(f"Contiguous free block: seats {fb[0]}–{fb[1]} (length {fb[2]})")
        else:
            print("Contiguous free block: none (sold out)")

        # In a real customer UI you might hide this map; here it helps you test.
        print("\nSeat map (RID per seat):")
        print(render_seat_map(sm))

        if reservations:
            df = pd.DataFrame([{
                "rid": r.rid,
                "name": r.name,
                "size": r.size,
                "created_at": r.created_at,
                "seats": f"{r.seats[0]}–{r.seats[-1]}" if r.seats else ""
            } for r in sorted(reservations.values(), key=lambda x: x.rid)])
            display(df)
        else:
            print("\nReservations: (none)")


def show_internal_view():
    with internal_out:
        clear_output()
        sm = seat_map_from_assign({rid: r.seats[:] for rid, r in reservations.items()})
        fb = free_block(sm)

        print("LIVE SEAT MAP")
        print(render_seat_map(sm))
        print()

        if fb[2] == 0:
            print("Free block: none (sold out)")
        else:
            print(f"Free block: seats {fb[0]}–{fb[1]} (length {fb[2]})")
        print()

        print("CURRENT RESERVATIONS")
        if reservations:
            df = pd.DataFrame([{
                "rid": r.rid,
                "name": r.name,
                "size": r.size,
                "created_at": r.created_at,
                "seats": f"{r.seats[0]}–{r.seats[-1]}" if r.seats else ""
            } for r in sorted(reservations.values(), key=lambda x: x.rid)])
            display(df)
        else:
            print("(none)")
        print("\n" + "="*70 + "\n")

        print("LAST COMPACTION EVENT")
        if event_log:
            ev = event_log[0]
            print(f"{ev['timestamp']} — {ev['action']} — {ev['meta']}\n")
            print("Before:")
            print(render_seat_map(ev["before_sm"]))
            print("\nAfter:")
            print(render_seat_map(ev["after_sm"]))

            if ev["moves"]:
                mv_df = pd.DataFrame([{
                    "rid": m["rid"],
                    "old": "" if m["old"] is None else (f"{m['old'][0]}–{m['old'][-1]}" if m["old"] else ""),
                    "new": "" if m["new"] is None else (f"{m['new'][0]}–{m['new'][-1]}" if m["new"] else ""),
                } for m in ev["moves"]]).sort_values("rid")
                print("\nWho moved:")
                display(mv_df)
            else:
                print("\nNo one moved (already compact).")
        else:
            print("(no events yet)")

        print("\nEVENT LOG (latest 10)")
        if event_log:
            log_df = pd.DataFrame([{
                "time": ev["timestamp"],
                "action": ev["action"],
                "meta": str(ev["meta"]),
                "moved_count": len(ev["moves"])
            } for ev in event_log[:10]])
            display(log_df)


def refresh_all(message: str = ""):
    refresh_cancel_dropdown()
    show_customer_view(message)
    show_internal_view()


# ---------- Button handlers ----------
def on_book(_):
    ok, msg = book(cust_name.value, cust_size.value)
    refresh_all(("✅ " if ok else "❌ ") + msg)

def on_cancel(_):
    rid = cancel_dropdown.value
    if rid is None:
        refresh_all("❌ Choose a reservation to cancel.")
        return
    ok, msg = cancel(rid)
    refresh_all(("✅ " if ok else "❌ ") + msg)

def on_reset(_):
    reset_all()
    refresh_all("✅ Reset complete.")


book_btn.on_click(on_book)
cancel_btn.on_click(on_cancel)
reset_btn.on_click(on_reset)

# Layout for Customer tab
customer_panel = widgets.VBox([
    widgets.HTML("<h3>Customer Booking Page</h3><p>No seat choice. System auto-compacts from seat 1.</p>"),
    widgets.HBox([cust_name, cust_size]),
    widgets.HBox([book_btn, reset_btn]),
    widgets.HBox([cancel_dropdown, cancel_btn]),
    cust_out
])

# Layout for Internal tab
internal_panel = widgets.VBox([
    widgets.HTML("<h3>Internal System View</h3><p>Shows how seats change after each book/cancel to prevent scattering.</p>"),
    internal_out
])

tab.children = [customer_panel, internal_panel]

# Initial render
refresh_all("Ready.")
display(tab)
